# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

In [2]:
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc, asc, sum as Fsum
from pyspark.sql.functions import udf, array_except
from pyspark.sql import Row
from pyspark.sql import Window
import numpy as np


In [3]:
spark = SparkSession\
        .builder\
        .appName("Wrangling Data")\
        .getOrCreate()

path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [4]:
user_log.head()

Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046')

# Question 1

Which page did user id "" (empty string) NOT visit?

In [5]:
# TODO: write your code to answer question 1
col1 = user_log.where(user_log.userId=="").select("page").dropDuplicates()
col2 = user_log.select("page").dropDuplicates().sort("page")
df = spark.createDataFrame([Row(c1=list(col1.toPandas()['page']),
                                c2=list(col2.toPandas()['page']))])
print("The pages not visited by id='' are: ")
df.select(array_except(df.c2, df.c1)).collect()

The pages not visited by id='' are: 


[Row(array_except(c2, c1)=['Downgrade', 'Error', 'Logout', 'NextSong', 'Save Settings', 'Settings', 'Submit Downgrade', 'Submit Upgrade', 'Upgrade'])]

# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [6]:
# TODO: use this space to explore the behavior of the user with an empty string
print(user_log.where(user_log.userId == "").select('auth').dropDuplicates().collect())
print(user_log.where(user_log.userId == "").select('page').dropDuplicates().collect())
user_log.where((user_log.userId == "") & (user_log.sessionId==3951)).collect()

[Row(auth='Logged Out'), Row(auth='Guest')]
[Row(page='Home'), Row(page='About'), Row(page='Login'), Row(page='Help')]


[Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=0, lastName=None, length=None, level='paid', location=None, method='GET', page='Home', registration=None, sessionId=3951, song=None, status=200, ts=1513745727284, userAgent=None, userId=''),
 Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=1, lastName=None, length=None, level='paid', location=None, method='GET', page='Home', registration=None, sessionId=3951, song=None, status=200, ts=1513746544284, userAgent=None, userId=''),
 Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=2, lastName=None, length=None, level='paid', location=None, method='GET', page='Home', registration=None, sessionId=3951, song=None, status=200, ts=1513746594284, userAgent=None, userId=''),
 Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=3, lastName=None, length=None, level='paid', location=None, method='GET', page='About', registration=None, 

# Question 3

How many female users do we have in the data set?

In [7]:
# TODO: write your code to answer question 3
user_log.select(['userId','gender']).dropDuplicates().groupby('gender').count().show()

+------+-----+
|gender|count|
+------+-----+
|     F|  462|
|  null|    1|
|     M|  501|
+------+-----+



# Question 4

How many songs were played from the most played artist?

In [8]:
# TODO: write your code to answer question 4

artist = user_log.select(['artist'])\
                 .where(user_log.artist != 'None')\
                 .groupby('artist')\
                 .count()\
                 .orderBy(desc('count'))\
                 .collect()[0]['artist']

print("The most played artist is: "+artist)

print("The number of songs played by "+artist+":")

user_log.select(['song','artist'])\
        .where(user_log.artist == artist)\
        .select(['song'])\
        .count()

The most played artist is: Coldplay
The number of songs played by Coldplay:


83

# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [9]:
visited_home_event = udf(lambda x: 1 if x == "Home" else 0,
                           IntegerType())

user_log = user_log.withColumn("VisitedHome",
                               visited_home_event("page"))

In [10]:
# TODO: write your code to answer question 5



In [11]:
windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)
user_log = user_log.withColumn("phase", Fsum("VisitedHome").over(windowval))

In [12]:
user_log.select(['auth','itemInSession',
                 'page','sessionId','ts',
                 'song','userId','VisitedHome','phase'])\
        .where(user_log.userId == '1046')\
        .sort("ts")\
        .collect()

[Row(auth='Logged In', itemInSession=112, page='NextSong', sessionId=5132, ts=1513720872284, song='Christmas Tears Will Fall', userId='1046', VisitedHome=0, phase=2),
 Row(auth='Logged In', itemInSession=113, page='NextSong', sessionId=5132, ts=1513721104284, song='Be Wary Of A Woman', userId='1046', VisitedHome=0, phase=2),
 Row(auth='Logged In', itemInSession=114, page='NextSong', sessionId=5132, ts=1513721310284, song='Public Enemy No.1', userId='1046', VisitedHome=0, phase=2),
 Row(auth='Logged In', itemInSession=115, page='NextSong', sessionId=5132, ts=1513721591284, song='Reign Of The Tyrants', userId='1046', VisitedHome=0, phase=2),
 Row(auth='Logged In', itemInSession=116, page='NextSong', sessionId=5132, ts=1513721808284, song='Father And Son', userId='1046', VisitedHome=0, phase=2),
 Row(auth='Logged In', itemInSession=117, page='NextSong', sessionId=5132, ts=1513721976284, song='No. 5', userId='1046', VisitedHome=0, phase=2),
 Row(auth='Logged In', itemInSession=118, page='N

In [20]:
user_log.select(['song','userId','phase'])\
        .where(user_log.song != 'None')\
        .groupby('userId','phase')\
        .count()\
        .where(user_log.userId!='')\
        .agg({"count" : "avg"})\
        .collect()

[Row(avg(count)=6.898347107438017)]